In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from cProfile import label #?not sure
import re
from fuzzywuzzy import fuzz
import difflib 
from fuzzywuzzy import process
import time
from tqdm import tqdm 
import fitz

from functools import reduce

### importing books

In [7]:
vol1_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 1.pdf'
vol2_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 2.pdf'
vol3_path = '../input/NOUVELLE FLORE DU LIBAN ET DE LA SYRIE 3.pdf'

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

vol1_pages = [vol1_doc[i] for i in range(vol1_doc.page_count)]
vol2_pages = [vol2_doc[i] for i in range(vol2_doc.page_count)]
vol3_pages = [vol3_doc[i] for i in range(vol3_doc.page_count)]

In [8]:
vol1_char_df = pd.read_pickle("../input/char_df/vol1_df.pkl")
vol2_char_df = pd.read_pickle("../input/char_df/vol2_df.pkl")
vol3_char_df = pd.read_pickle("../input/char_df/vol3_df.pkl")

In [9]:
#mout. index: 
vol1_index_path = '../output/local/index_output/vol1_nonitalics.csv'
vol2_index_path = '../output/local/index_output/vol2_nonitalics.csv'
vol3_index_path = '../output/local/index_output/vol3_nonitalics.csv'

vol1_index_df = pd.read_csv(vol1_index_path)
vol2_index_df = pd.read_csv(vol2_index_path)
vol3_index_df = pd.read_csv(vol3_index_path)

#changing name of columns of mout. indecies 
vol1_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol2_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)
vol3_index_df.rename(columns={'closest_genus': 'mouterde_genus', 'closest_epithet': 'mouterde_epithet', 'authors':'mouterde_author', 'closest_infra_name':'mouterde_infra'}, inplace=True)

In [10]:
vol1_word_df = vol1_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

vol2_word_df = vol2_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

vol3_word_df = vol3_char_df.loc[:, ['vol_num', 'page_num', 
                                    'block_num', 'block_num_absolute', 'block_bbox',
                                    'line_num', 'line_wmode', 'line_dir', 'line_bbox', 
                                    'span_num', 'span_size', 'span_flags', 'span_font', 'span_color', 'span_ascender', 'span_descender', 'span_origin', 'span_bbox', 
                                    'word_num', 'word','word_bbox', 'pruned_word', 'pruned_word_bbox']].drop_duplicates()

### getting word pairs

*right now last word of page i matches with alst word of page i + 1*

things that I need to account for and am not doing so right now: 
- what if a word is in the next paragraph/block 
- what if a word is in the next page 

In [27]:
#list of genera from index -- uppercased to match main text pattern
vol1_genera = vol1_index_df[vol1_index_df['taxon_rank'] == 'genus']['mouterde_genus'].str.upper().tolist()

#list of species binomial from main text
vol1_species_temp_df = vol1_index_df[(vol1_index_df['taxon_rank'] == 'epithet') & (~vol1_index_df['mouterde_genus'].isna())]
vol1_species_binomial_list = list(zip(vol1_species_temp_df['mouterde_genus'], vol1_species_temp_df['mouterde_epithet']))
vol1_species = list(map(lambda x: f"{x[0]} {x[1]}", vol1_species_binomial_list))
vol1_species_abriviation = list(map(lambda x: f"{x[0][0]}. {x[1]}", vol1_species_binomial_list))

In [103]:
def is_italic(flags):
    return flags & 2 ** 1 != 0

In [29]:
def get_n_words_flagged(df, n, inplace = True):
    #assumes n > 1
    out_words_col = f"{n}_words"
    out_flags_col = f"{n}_flags"
    
    line_group_cols = ['vol_num', 'page_num', 
                       'block_num', 'block_num_absolute', 'block_bbox', 
                       'line_num', 'line_wmode', 'line_dir', 'line_bbox']
    
    n_words_lists = [None for i in range(n)]
    n_words_flags = [None for i in range(n)]

    n_words_lists[0] = df['word']
    n_words_flags[0] = df['span_flags']

    for i in range(1, n):
        n_words_lists[i] = df.groupby(line_group_cols)['word'].shift(-i, fill_value="")
        n_words_flags[i] = df.groupby(line_group_cols)['span_flags'].shift(-i, fill_value=0)

    zip_n_words = list(zip(*n_words_lists))
    n_word_string = list(map(lambda n_word_list : " ".join(n_word_list), zip_n_words))

    zip_n_flags = list(zip(*n_words_flags))
    combine_flags = list(map(lambda flag_list : reduce(lambda x, y: x | y, flag_list), zip_n_flags))
    
    if inplace == True:
        df[out_words_col] = n_word_string
        df[out_flags_col] = combine_flags
    return n_word_string, combine_flags

In [30]:
tqdm.pandas()

In [25]:
def difflib_closest_matches(input_str, matching_lst):
    matched_list = difflib.get_close_matches(input_str, matching_lst)
    if len(matched_list) > 0:
        return matched_list[0]
    else:
        #print(input_str)
        return np.NaN

In [31]:
def difflib_closest_match_score(input_str, match_str):
    if isinstance(match_str, str):
        input_str = input_str.replace(' ', '')
        match_str = match_str.replace(' ', '')
        score = difflib.SequenceMatcher(None, input_str.lower(), match_str.lower()).ratio()
        return score
    else:
        return np.NaN

In [34]:
for n in range(1, 5):
    word_group_col_name = f"{n}_words"
    match_col_name = f"{n}_words_match"
    match_score_col_name = f"{n}_words_match_score"
    print(f"finding best match for {word_group_col_name}")
    get_n_words_flagged(vol1_word_df, n)
    vol1_word_df[match_col_name] = vol1_word_df[word_group_col_name].progress_apply(lambda x: difflib_closest_matches(x, vol1_species))
    vol1_word_df[match_score_col_name] = vol1_word_df.progress_apply(lambda r: difflib_closest_match_score(r[word_group_col_name], r[match_col_name]), axis = 1)

finding best match for 1_words


100%|██████████| 316313/316313 [00:01<00:00, 177623.73it/s]


finding best match for 2_words


100%|██████████| 316313/316313 [00:01<00:00, 173642.59it/s]


finding best match for 3_words


100%|██████████| 316313/316313 [00:01<00:00, 176867.70it/s]


finding best match for 4_words


100%|██████████| 316313/316313 [00:01<00:00, 183662.32it/s]


In [50]:
vol1_word_df[vol1_word_df['word'] == 'Pennisetum']

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,4_words,4_flags,1_words_match,1_words_match_score,2_words_match,2_words_match_score,3_words_match,3_words_match_score,4_words_match,4_words_match_score
300771,1,115,24,57,"(64.80000305175781, 455.3900146484375, 437.058...",0,0,"(1.0, 0.0)","(64.80000305175781, 455.3900146484375, 437.058...",0,...,Pennisetum setaceum (Forsk.) Chiov.,20,NaN,NaN,Penmsetum setaceum,0.914286,Penmsetum setaceum,0.744186,Penmsetum setaceum,0.653061
302863,1,116,15,34,"(41.7599983215332, 294.75, 405.2952575683594, ...",0,0,"(1.0, 0.0)","(41.7599983215332, 294.75, 405.2952575683594, ...",2,...,Pennisetum ciliare (L.) Link,6,NaN,NaN,Penmsetum divisum,0.606061,NaN,NaN,NaN,NaN


In [352]:
difflib_closest_match_score('Pennisetum divisum','Penmsetum divisum')

0.9090909090909091

In [70]:
is_italic

<function __main__.is_italic(flags)>

In [69]:
vol1_word_df['1_flags'].apply(is_italic)

0          False
6          False
14         False
16         False
20         False
           ...  
1761774    False
1761781    False
1761784    False
1761788    False
1761792    False
Name: 1_flags, Length: 316313, dtype: bool

In [ ]:
['4_flags'].apply(is_italic)

In [88]:
# things to add: 
# make sure they're on the same line (fixing the shift situation by doing it on group by?)
# make sure single word match is nothing completely unreasonable (e.g > 0.1 or something like that)
# make sure no words in the grouping are italics 
likely_results = vol1_word_df[~((vol1_word_df['1_flags'].apply(is_italic)) | 
                                (vol1_word_df['2_flags'].apply(is_italic)) |
                                (vol1_word_df['3_flags'].apply(is_italic)) | 
                                (vol1_word_df['4_flags'].apply(is_italic))) &
                              ((vol1_word_df['1_words_match_score'] > 0.85) | 
                               (vol1_word_df['2_words_match_score'] > 0.85) | 
                               (vol1_word_df['3_words_match_score'] > 0.85) | 
                               (vol1_word_df['4_words_match_score'] > 0.85)) & 
                              (vol1_word_df['page_num'] < 616)]

In [104]:
# things to add: 
# make sure they're on the same line (fixing the shift situation by doing it on group by?)
# make sure single word match is nothing completely unreasonable (e.g > 0.1 or something like that)
# make sure no words in the grouping are italics 
likely_results = vol1_word_df[((~(vol1_word_df['1_flags'].apply(is_italic)) & 
                                (vol1_word_df['1_words_match_score'] > 0.85)) |
                               (~(vol1_word_df['2_flags'].apply(is_italic)) & 
                                (vol1_word_df['2_words_match_score'] > 0.85)) |
                               (~(vol1_word_df['3_flags'].apply(is_italic)) & 
                                (vol1_word_df['3_words_match_score'] > 0.85)) |
                               (~(vol1_word_df['1_flags'].apply(is_italic)) & 
                                (vol1_word_df['1_words_match_score'] > 0.85))) &
                              (vol1_word_df['page_num'] < 616)]

In [105]:
from fitz.utils import getColor

In [106]:
# reset doc:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

In [107]:
for i in tqdm(likely_results.index):
    page_num = int(likely_results.loc[i, 'page_num'])
    block_bbox = likely_results.loc[i, 'block_bbox']
    block_bbox = likely_results.loc[i, 'block_bbox']
    word_bbox = likely_results.loc[i, 'word_bbox']
    page = vol1_doc[page_num]
    r_high = fitz.Rect(word_bbox)
    r_box = fitz.Rect(block_bbox)
    annot_high = page.add_highlight_annot(r_high)
    annot_high.set_colors(stroke=getColor("antiquewhite"))
    annot_rect = page.add_rect_annot(r_box)
    annot_high.update()
    annot_rect.update()

marked_epithet_fname = "../output/local/main_text/fuzzyMatch85_vol1_newNoIv2.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 1096/1096 [00:00<00:00, 4958.88it/s]


### problems

Not being picked up: 
118 - Erianthusravennae - Index Issue 
118 - Erianthus hostii - Index Issue
140 - Alopecurus my o sur oi des - spaces, would need 6_words which would be O(crazy)
172 - Molinia caemlea - Index Issue - index is: Molinia oerulea (match score: 0.785714)
247 - Fuirena pub e seen s - NOT SURE! EVERYTHING SEEMS FINE 4_words_match_score = 0.9!
268 - A r u m melanopus -  spaces, would need 5_words which would be O(crazy)
306 - Tulipa aff. biflorae - Q: aff. ? and matches with Tulipa biflora
348 - Allium deserti-syriaci - Matches with Allium de∫er˛i
501 - Hypocylix sp.? - Notation doesn't make sense
516 - Halocharis sp. - Notation doesn't make sense
549 - Arenaria libanotica - Index Issue (Arenaria liban otica Ky - 2 match: 0.787879) 

588 - Gypsophila venusta Fenzl - Index Issue (is in online version but not offline)
590 - Gypsophila libanotica - Index Issue (is in online version but not offline)
590 - Gypsophila pallida - Index Issue (is in online version but not offline)
591 - Gypsophila ruscifolia - Index Issue (is in online version but not offline)
591 - Gypsophila linearifolia - Index Issue (is in online version but not offline)
...
594 - Gypsophila ... Index Issue (is in online version but not offline) [11 total]

__
Unrelated:
S. 510, 511
Box overlap : 175 
Box stop early? 207, 222 & 578 (just sp. name), 250 & 263 (prev cond -- issue visible after page 1)

In [122]:
vol1_word_df[vol1_word_df['2_words'] == 'Allium deserti-syriaci'].iloc[:,20:]

,word_bbox,pruned_word,pruned_word_bbox,1_words,1_flags,2_words,2_flags,3_words,3_flags,4_words,4_flags,1_words_match,1_words_match_score,2_words_match,2_words_match_score,3_words_match,3_words_match_score,4_words_match,4_words_match_score
956721,"(44.400001525878906, 414.19000244140625, 79.88...",Allium,"(44.400001525878906, 414.19000244140625, 79.88...",Allium,20,Allium deserti-syriaci,20,"Allium deserti-syriaci Feinbr.,",20,"Allium deserti-syriaci Feinbr., loc.",22,Allium cepa,0.75,Allium de∫er˛i,0.647059,NaN,NaN,NaN,NaN


In [123]:
vol1_word_df[vol1_word_df['2_words'] == 'Arenaria libanotica'].iloc[:,20:]

,word_bbox,pruned_word,pruned_word_bbox,1_words,1_flags,2_words,2_flags,3_words,3_flags,4_words,4_flags,1_words_match,1_words_match_score,2_words_match,2_words_match_score,3_words_match,3_words_match_score,4_words_match,4_words_match_score
1528694,"(44.880001068115234, 330.5899963378906, 90.449...",Arenaria,"(44.880001068115234, 330.5899963378906, 90.449...",Arenaria,20,Arenaria libanotica,20,Arenaria libanotica Ky,20,Arenaria libanotica Ky (incl.,20,Arenaria liban,0.761905,Arenaria liban,0.83871,Arenaria liban,0.787879,Minuartia libanotica,0.622222


In [125]:
vol1_word_df[vol1_word_df['3_words'] == 'Gypsophila venusta Fenzl'].iloc[:,20:]

,word_bbox,pruned_word,pruned_word_bbox,1_words,1_flags,2_words,2_flags,3_words,3_flags,4_words,4_flags,1_words_match,1_words_match_score,2_words_match,2_words_match_score,3_words_match,3_words_match_score,4_words_match,4_words_match_score
1636313,"(54.720001220703125, 531.9100341796875, 112.96...",Gypsophila,"(54.720001220703125, 531.9100341796875, 112.96...",Gypsophila,20,Gypsophila venusta,20,Gypsophila venusta Fenzl,20,Gypsophila venusta Fenzl (PI.,20,Gypsophila antari,0.769231,Gypsophila antari,0.787879,Gypsophila antari,0.684211,Gypsophila antari,0.666667


In [127]:
vol1_index_df[vol1_index_df['mouterde_genus'] == 'Gypsophila']

,Unnamed: 0,mouterde_genus,mouterde_epithet,mouterde_infra,mouterde_author,taxon_rank,taxon_rank_detailed
615,2466,Gypsophila,NaN,NaN,NaN,genus,genus
616,2467,Gypsophila,anatolica,NaN,Boiss.,epithet,epithet
617,2469,Gypsophila,antari,NaN,Post et Beauv.,epithet,epithet
618,2475,Gypsophila,arabica,NaN,Barkoudah,epithet,epithet
619,2480,Gypsophila,aucheri,NaN,Boiss.,epithet,epithet
620,2488,Gypsophila,damascena,NaN,Boiss.,epithet,epithet
621,2493,Gypsophila,filicaulis,NaN,NaN,epithet,epithet


In [112]:
vol1_word_df[vol1_word_df['word'] == 'Molinia'].iloc[:,20:]

,word_bbox,pruned_word,pruned_word_bbox,1_words,1_flags,2_words,2_flags,3_words,3_flags,4_words,4_flags,1_words_match,1_words_match_score,2_words_match,2_words_match_score,3_words_match,3_words_match_score,4_words_match,4_words_match_score
456905,"(65.5199966430664, 152.1099853515625, 104.4586...",Molinia,"(65.5199966430664, 152.1099853515625, 104.4586...",Molinia,20,Molinia caemlea,20,Molinia caemlea (L.),20,Molinia caemlea (L.) Moench,20,Molinia oerulea,0.666667,Molinia oerulea,0.785714,Molinia oerulea,0.687500,NaN,NaN
1741373,"(28.559999465942383, 240.92999267578125, 58.67...",Molinia,"(28.559999465942383, 240.92999267578125, 58.67...",Molinia,4,Molinia,4,Molinia,4,Molinia,4,Molinia oerulea,0.666667,Molinia oerulea,0.666667,Molinia oerulea,0.666667,Molinia oerulea,0.666667


### end points 

In [128]:
# reset doc:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

In [164]:
first_quad = (vol1_word_df['line_bbox'].apply(lambda x: x[0]).min() + vol1_word_df['line_bbox'].apply(lambda x: x[2]).max())/4

150.6609063707292

In [170]:
vol1_word_df[(vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & (vol1_word_df['word'].str.isupper())]

,vol_num,page_num,block_num,block_num_absolute,block_bbox,line_num,line_wmode,line_dir,line_bbox,span_num,...,4_words,4_flags,1_words_match,1_words_match_score,2_words_match,2_words_match_score,3_words_match,3_words_match_score,4_words_match,4_words_match_score
486,1,1,2,2,"(215.52000427246094, 218.25149536132812, 232.4...",0,0,"(1.0, 0.0)","(215.52000427246094, 218.25149536132812, 232.4...",0,...,PAR,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
489,1,1,3,3,"(151.1999969482422, 239.97000122070312, 296.83...",0,0,"(1.0, 0.0)","(151.1999969482422, 239.97000122070312, 296.83...",0,...,"PAUL MOUTERDE, S.J.",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
493,1,1,3,3,"(151.1999969482422, 239.97000122070312, 296.83...",0,0,"(1.0, 0.0)","(151.1999969482422, 239.97000122070312, 296.83...",0,...,"MOUTERDE, S.J.",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
502,1,1,3,3,"(151.1999969482422, 239.97000122070312, 296.83...",0,0,"(1.0, 0.0)","(151.1999969482422, 239.97000122070312, 296.83...",0,...,S.J.,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,1,1,4,4,"(166.55999755859375, 323.9100036621094, 281.38...",0,0,"(1.0, 0.0)","(166.55999755859375, 323.9100036621094, 281.38...",0,...,TOME PREMIER,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1761774,1,641,4,6,"(154.0800018310547, 284.6300048828125, 227.289...",0,0,"(1.0, 0.0)","(154.0800018310547, 284.6300048828125, 227.289...",0,...,JUILLET,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1761781,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,MIL NEUF CENT SOIXANTE-SIX,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1761784,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,NEUF CENT SOIXANTE-SIX,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1761788,1,641,5,7,"(134.63999938964844, 294.22998046875, 245.3221...",0,0,"(1.0, 0.0)","(134.63999938964844, 294.22998046875, 245.3221...",0,...,CENT SOIXANTE-SIX,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [174]:
#possible_stops = vol1_word_df[((vol1_word_df['word'].str.isupper()) & (vol1_word_df['pruned_word'].apply(len) > 2)) | (vol1_word_df['word'].isin(vol1_genera))]
genus_parts = vol1_word_df[(vol1_word_df['word'].isin(vol1_genera))] #or technically doing a close match etc 
middle_uppers = vol1_word_df[(vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & (vol1_word_df['word'].str.isupper())]
possible_stops = vol1_word_df[((vol1_word_df['word'].isin(vol1_genera))) | ((vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & (vol1_word_df['word'].str.isupper()) & (vol1_word_df['pruned_word'].apply(len) > 2))]

In [175]:
# reset doc:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

In [176]:
#possible_stops = genus_parts 
for i in tqdm(possible_stops.index):
    page_num = int(possible_stops.loc[i, 'page_num'])
    block_bbox = possible_stops.loc[i, 'block_bbox']
    block_bbox = possible_stops.loc[i, 'block_bbox']
    word_bbox = possible_stops.loc[i, 'word_bbox']
    page = vol1_doc[page_num]
    r_high = fitz.Rect(word_bbox)
    r_box = fitz.Rect(block_bbox)
    annot_high = page.add_highlight_annot(r_high)
    annot_high.set_colors(stroke=getColor("antiquewhite"))
    annot_rect = page.add_rect_annot(r_box)
    annot_high.update()
    annot_rect.update()

marked_epithet_fname = "../output/local/main_text/stop_v2.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 1803/1803 [00:00<00:00, 5685.06it/s]


In [184]:
vol1_word_df['line_id'] = vol1_word_df.progress_apply(lambda r : (r['page_num'], r['block_num'], r['line_num']), axis = 1) 

100%|██████████| 316313/316313 [00:01<00:00, 166711.11it/s]


In [190]:
is_binomial = ((~(vol1_word_df['1_flags'].apply(is_italic)) & (vol1_word_df['1_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['2_flags'].apply(is_italic)) & (vol1_word_df['2_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['3_flags'].apply(is_italic)) & (vol1_word_df['3_words_match_score'] > 0.85)) | 
               (~(vol1_word_df['1_flags'].apply(is_italic)) & (vol1_word_df['1_words_match_score'] > 0.85))) 

is_stop = (((vol1_word_df['word'].isin(vol1_genera))) | 
           ((vol1_word_df['line_bbox'].apply(lambda x : x[0] > 120)) & 
            (vol1_word_df['word'].str.isupper()) & 
            (vol1_word_df['pruned_word'].apply(len) > 2)))

break_page_num = vol1_word_df[(is_binomial) | (is_stop)]['page_num']
break_block_num = vol1_word_df[(is_binomial) | (is_stop)]['block_num']
break_line_num = vol1_word_df[(is_binomial) | (is_stop)]['line_num']
break_id = list(zip(break_page_num, break_block_num, break_line_num))
vol1_word_df['section_break'] = vol1_word_df['line_id'].isin(break_id)

In [195]:
def get_section_id(row):
    if row['section_break'] == True:
        return row['line_id']
    else:
        return np.nan
vol1_word_df['section_id'] = vol1_word_df.progress_apply(get_section_id, axis = 1)

100%|██████████| 316313/316313 [00:01<00:00, 258685.41it/s]


In [199]:
vol1_word_df['section_id'].ffill(inplace=True)

In [216]:
def get_section_start_y(row):
    if row['section_break'] == True:
        return row['line_bbox'][1]
    else:
        return np.nan
# ssection break start y ... making sure we didn't "go up" in ffil
vol1_word_df['section_start_y'] = vol1_word_df.progress_apply(get_section_start_y, axis = 1)
vol1_word_df['section_start_y'].ffill(inplace=True)

100%|██████████| 316313/316313 [00:01<00:00, 256695.08it/s]


In [ ]:
def get_section_start_y(row):
    if row['section_break'] == True:
        return row['line_bbox'][1]
    else:
        return np.nan
# ssection break start y ... making sure we didn't "go up" in ffil
vol1_word_df['section_start_y'] = vol1_word_df.progress_apply(get_section_start_y, axis = 1)
vol1_word_df['section_start_y'].ffill(inplace=True)

In [219]:
# break down line coords
vol1_word_df['line_x0'] = vol1_word_df["line_bbox"].apply(lambda x: x[0])
vol1_word_df['line_y0'] = vol1_word_df["line_bbox"].apply(lambda x: x[1])
vol1_word_df['line_x1'] = vol1_word_df["line_bbox"].apply(lambda x: x[2])
vol1_word_df['line_y1'] = vol1_word_df["line_bbox"].apply(lambda x: x[3])

#sections_coords: 
vol1_word_df["section_x0"] = vol1_word_df.groupby('section_id')['line_x0'].transform('min')
vol1_word_df["section_y0_all"] = vol1_word_df.groupby('section_id')['line_y0'].transform('min')
vol1_word_df["section_y0"] = vol1_word_df[['section_y0_all','section_start_y']].max(axis=1)
vol1_word_df["section_x1"] = vol1_word_df.groupby('section_id')['line_x1'].transform('max')
vol1_word_df["section_y1"] = vol1_word_df.groupby('section_id')['line_y1'].transform('max')

#section_bbox:
vol1_word_df["section_bbox"] = vol1_word_df.apply(lambda r: (r["section_x0"], r["section_y0"], r["section_x1"], r["section_y1"]), axis = 1)

#drop extra cols:
vol1_word_df.drop(columns= ["line_x0", "line_y0", "line_x1", "line_y1", "section_x0", "section_y0", "section_x1", "section_y1"], inplace = True)

In [220]:
# reset doc:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

In [206]:
# section_groups = vol1_word_df.groupby('section_id')
# for name, section in tqdm(section_groups):
#     #for each group in the section ids I need to do the following:
#     page_num = int(vol1_word_df.loc[i, 'page_num'])
#     # block_bbox = vol1_word_df.loc[i, 'block_bbox']
#     # block_bbox = vol1_word_df.loc[i, 'block_bbox']
#     section_bbox = vol1_word_df.loc[i, 'section_bbox']
#     page = vol1_doc[page_num]
#     r_box = fitz.Rect(section_bbox)
#     #annot_high = page.add_highlight_annot(r_high)
#     #annot_high.set_colors(stroke=getColor("antiquewhite"))
#     annot_rect = page.add_rect_annot(r_box)
#     #annot_high.update()
#     annot_rect.update()


# marked_epithet_fname = "../output/local/main_text/sections_v1.pdf"
# vol1_doc.save(marked_epithet_fname)

100%|██████████| 316313/316313 [03:15<00:00, 1619.81it/s]


In [221]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

section_groups = vol1_word_df.groupby('section_id')
for name, section in tqdm(section_groups):
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_bbox = section.iloc[0]['section_bbox']
    r_box = fitz.Rect(section_bbox)
    annot_rect = page.add_rect_annot(r_box)
    annot_rect.update()

marked_epithet_fname = "../output/local/main_text/sections_v3.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 2228/2228 [00:00<00:00, 3319.53it/s]


In [222]:
binom_page_num = vol1_word_df[(is_binomial)]['page_num']
binom_block_num = vol1_word_df[(is_binomial)]['block_num']
binom_line_num = vol1_word_df[(is_binomial)]['line_num']
binom_id = list(zip(binom_page_num, binom_block_num, binom_line_num))
vol1_word_df['binom_section'] = vol1_word_df['line_id'].isin(binom_id)

In [224]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

section_groups = vol1_word_df.groupby('section_id')
for name, section in tqdm(section_groups):
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['section_id']
    section_bbox = section.iloc[0]['section_bbox']
    if section_id in binom_id:
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()

marked_epithet_fname = "../output/local/main_text/binom_sections_v0.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 2228/2228 [00:00<00:00, 5000.08it/s]


In [225]:
# and this is pretty great :)

In [240]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

section_groups = vol1_word_df.groupby('section_id')
prev_sec = None
for name, section in tqdm(section_groups):
    cont = section['word'].isin(['NOUVELLE', 'FLORE']).any()
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['section_id']
    section_bbox = section.iloc[0]['section_bbox']
    if section_id in binom_id or (cont and (prev_sec in binom_id)):
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    prev_sec = section_id
marked_epithet_fname = "../output/local/main_text/binom_sections_page_cont_v1.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 2228/2228 [00:00<00:00, 3505.32it/s]


In [ ]:
# prev cont -- not tested

vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)

section_groups = vol1_word_df.groupby('section_id')
prev_sec = None
prev_cont = True
for name, section in tqdm(section_groups):
    cont = section['word'].isin(['NOUVELLE', 'FLORE']).any()
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['section_id']
    section_bbox = section.iloc[0]['section_bbox']
    if section_id in binom_id or (cont and (prev_sec in binom_id)) or (cont and prev_cont):
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    if cont or prev_cont:
        prev_cont = True
    else: 
        prev_cont = False
    prev_sec = section_id

In [241]:
vol1_word_df.to_pickle("../input/desc_box_df/vol1_desc_df.pkl")

In [242]:
vol1_doc = fitz.open(vol1_path)
vol2_doc = fitz.open(vol2_path)
vol3_doc = fitz.open(vol3_path)
vol1_doc = fitz.open("/Users/shidedehghani/Library/Mobile Documents/com~apple~CloudDocs/Research/rLAB/Maryam's_repo/Aaron/results-book-1.pdf")
section_groups = vol1_word_df.groupby('section_id')
prev_sec = None
for name, section in tqdm(section_groups):
    cont = section['word'].isin(['NOUVELLE', 'FLORE']).any()
    page_num = int(name[0])
    page = vol1_doc[page_num]
    section_id = section.iloc[0]['section_id']
    section_bbox = section.iloc[0]['section_bbox']
    if section_id in binom_id or (cont and (prev_sec in binom_id)):
        r_box = fitz.Rect(section_bbox)
        annot_rect = page.add_rect_annot(r_box)
        annot_rect.update()
    prev_sec = section_id
marked_epithet_fname = "../output/local/main_text/arronXme_vol1_v1.pdf"
vol1_doc.save(marked_epithet_fname)

100%|██████████| 2228/2228 [00:00<00:00, 2718.45it/s]
